<a href="https://colab.research.google.com/github/DomizianoScarcelli/big-data-project/blob/nn-model/NN_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install dependencies

In [1]:
!pip install petastorm -qq
!pip install pyspark -qq
!pip install -U -q PyDrive -qq
!apt install openjdk-8-jdk-headless -qq

openjdk-8-jdk-headless is already the newest version (8u372-ga~us1-0ubuntu1~20.04).
0 upgraded, 0 newly installed, 0 to remove and 28 not upgraded.


In [2]:
#@title Imports
import os
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly

import pyspark
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, FloatType, LongType
import pyspark.sql.functions as F
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.ml.linalg import SparseVector, DenseVector

from tqdm.notebook import tqdm
import time
import gc

from google.colab import drive
from petastorm import make_batch_reader
from petastorm.pytorch import DataLoader

In [3]:
#@title Set up variables
JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64"
GDRIVE_DIR = "/content/drive"
GDRIVE_HOME_DIR = GDRIVE_DIR + "/MyDrive"
GDRIVE_DATA_DIR = GDRIVE_HOME_DIR + "/Big Data/datasets"
DATASET_FILE = os.path.join(GDRIVE_DATA_DIR, "pyspark_friendly_spotify_playlist_dataset")
AUDIO_FEATURES_FILE = os.path.join(GDRIVE_DATA_DIR, "pyspark_track_features")
LITTLE_SLICE_FILE = os.path.join(GDRIVE_DATA_DIR, "little_slice")
SMALL_SLICE_FLIE = os.path.join(GDRIVE_DATA_DIR, "small_slice")
LITTLE_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "little_slice_audio_features")
MICRO_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "micro_slice_audio_features")
SPLITTED_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "splitted_pyspark_track_features")
SAVED_DFS_PATH = os.path.join(GDRIVE_DATA_DIR, "saved_dfs")
RANDOM_SEED = 42 # for reproducibility
os.environ["JAVA_HOME"] = JAVA_HOME
os.environ["PYSPARK_PYTHON"]="python"

In [4]:
drive.mount(GDRIVE_DIR, force_remount=True)

Mounted at /content/drive


In [5]:
#@title Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '12G').\
                set('spark.driver.memory', '12G').\
                set('spark.driver.maxResultSize', '100G').\
                set("spark.executor.extraJavaOptions", "-XX:+UseG1GC").\
                setAppName("PySparkTutorial").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

## Setup ngrok

In [6]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!ngrok authtoken 2NVN8kdoOnMVtlDGGWtwsbT5M3Q_2EJv2HE77FEXkz978Qtnq

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [8]:
from pyngrok import ngrok

# Open a ngrok tunnel on the port 4050 where Spark is running
port = '4050'
public_url = ngrok.connect(port).public_url

In [9]:
print("To access the Spark Web UI console, please click on the following link to the ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"".format(public_url, port))

To access the Spark Web UI console, please click on the following link to the ngrok tunnel "https://c6a8-35-201-201-181.ngrok-free.app" -> "http://127.0.0.1:4050"


In [10]:
#@title Check if everything is ok
spark, sc._conf.getAll()


(<pyspark.sql.session.SparkSession at 0x7f251c5e75e0>,
 [('spark.executor.extraJavaOptions',
   '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false -XX:+UseG1GC'),
  ('spark.app.name', 'PySparkTutorial'),
  ('spark.driver.host', 'e8b508dba625'),


# Data acquisition

In [11]:
song_schema = StructType([
    StructField("pos", IntegerType(), True),
    StructField("artist_name", StringType(), True),
    StructField("track_uri", StringType(), True),
    StructField("artist_uri", StringType(), True),
    StructField("track_name", StringType(), True),
    StructField("album_uri", StringType(), True),
    StructField("duration_ms", LongType(), True),
    StructField("album_name", StringType(), True)
])

playlist_schema = StructType([
    StructField("name", StringType(), True),
    StructField("collaborative", StringType(), True),
    StructField("pid", IntegerType(), True),
    StructField("modified_at", IntegerType(), True),
    StructField("num_tracks", IntegerType(), True),
    StructField("num_albums", IntegerType(), True),
    StructField("num_followers", IntegerType(), True),
    StructField("tracks", ArrayType(song_schema), True),
    StructField("num_edits", IntegerType(), True),
    StructField("duration_ms", IntegerType(), True),
    StructField("num_artists", IntegerType(), True),
])

audio_features_schema = StructType([
    StructField("danceability", FloatType(), True),
    StructField("energy", FloatType(), True),
    StructField("key", IntegerType(), True),
    StructField("loudness", FloatType(), True),
    StructField("mode", IntegerType(), True),
    StructField("speechiness", FloatType(), True),
    StructField("acousticness", FloatType(), True),
    StructField("instrumentalness", FloatType(), True),
    StructField("liveness", FloatType(), True),
    StructField("valence", FloatType(), True),
    StructField("tempo", FloatType(), True),
    StructField("type", StringType(), True),
    StructField("id", StringType(), True),
    StructField("uri", StringType(), True),
    StructField("track_href", StringType(), True),
    StructField("analysis_url", StringType(), True),
    StructField("duration_ms", LongType(), True),
    StructField("time_signature", IntegerType(), True)
])


In [12]:
slice_df = spark.read.schema(playlist_schema).json(SMALL_SLICE_FLIE, multiLine=True)

# Import pyspark dataframe

In [13]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
from pyspark.ml.linalg import VectorUDT

playlist_schema_mapped = StructType([
    StructField("name", StringType(), True),
    StructField("collaborative", StringType(), True),
    StructField("pid", IntegerType(), True),
    StructField("modified_at", IntegerType(), True),
    StructField("num_tracks", IntegerType(), True),
    StructField("num_albums", IntegerType(), True),
    StructField("num_followers", IntegerType(), True),
    StructField("tracks", VectorUDT(), True),
    StructField("num_edits", IntegerType(), True),
    StructField("duration_ms", IntegerType(), True),
    StructField("num_artists", IntegerType(), True),
])

In [15]:
PLAYLIST_EMBEDDINGS = os.path.join(SAVED_DFS_PATH, "playlist_embeddings_new.parquet")
JSON_PLAYLIST_EMBEDDINGS = os.path.join(SAVED_DFS_PATH, "playlist_embeddings_NN.json")

In [16]:
# if os.path.exists(JSON_PLAYLIST_EMBEDDINGS):
#   mapped_slice_df = spark.read.schema(playlist_schema_mapped).json(JSON_PLAYLIST_EMBEDDINGS)
# else:
#   mapped_slice_df = spark.read.schema(playlist_schema_mapped).parquet(PLAYLIST_EMBEDDINGS)
#   mapped_slice_df.write.json(JSON_PLAYLIST_EMBEDDINGS)

# Extract artist matrix $\mathbf{A}$

From the main dataframe `slice_df` that contains the information about playlists and their relative songs, I want to obtain an artist binary `SparseVector` $\mathbf{a}$ for each playlist that describes the artists that are inside of that playlist.
Stacked togheter, all the vectors make the artists matrix $\mathbf{A} \in \mathbb{R}^{m \times k}$

In [17]:
slice_df = slice_df.limit(10_000).cache()

In [18]:
# slice_df.limit(1).cache().select(explode("tracks")).select("col.*").show()

In [19]:
from typing import Tuple
from pyspark.sql import DataFrame
from pyspark.ml.linalg import VectorUDT
from pyspark.sql.functions import col, udf, explode
from pyspark.sql.types import IntegerType, ArrayType
from functools import reduce

def get_all_artists(playlist_df: DataFrame) -> Tuple[DataFrame, int]:
   all_songs = playlist_df.select(explode("tracks.artist_uri").alias("artist_uri")).distinct()
   return all_songs

def create_artists_pos_mapping(artists_df: DataFrame) -> DataFrame:
  artists_df = get_all_artists(slice_df)
  artists_df.createOrReplaceTempView("ARTISTS")
  artists_df = spark.sql("""
  SELECT 
      row_number() OVER (
          PARTITION BY '' 
          ORDER BY '' 
      ) as pos,
      *
  FROM 
      ARTISTS
  """)

  artists_df = artists_df.sort("artist_uri")

  ARTIST_VECTOR_LENGTH = artists_df.count()

  return artists_df, ARTIST_VECTOR_LENGTH


#TODO: Since the .rdd is very slow, I can embed the position information of the track inside the track itself,
# So then I can just do pos_list.add(row.rating_position) in a few miliseconds. 
def create_artists_vector(playlist_df: DataFrame, mapping: DataFrame) -> DataFrame:
    """
    Returns a DataFrames containing the playlists, but the tracks are represented as a binary sparse vector.
    """

    @udf(returnType=VectorUDT())
    def extract_vector(tracks):
      pos_list = set()

      def reduce_fn(pos_list, row):
          pos_list.add(artist_uri_to_id.get(row.artist_uri))
          return pos_list
      
      pos_list = reduce(reduce_fn, tracks, pos_list)
      
      return SparseVector(ARTIST_VECTOR_LENGTH + 1, sorted(list(pos_list)), [1 for _ in pos_list])

    # Apply the mapping UDF on the "tracks" column of the slice_df dataframe
    mapped_df = playlist_df.withColumn('tracks', extract_vector(col('tracks')))

    return mapped_df

artists_df, ARTIST_VECTOR_LENGTH = create_artists_pos_mapping(slice_df)
artist_uri_to_id = artists_df.select('artist_uri', 'pos').rdd.collectAsMap() # TODO: Pass it as a parameter maybe?
# if not os.path.exists(PLAYLIST_EMBEDDINGS):
artists_slice_df = create_artists_vector(slice_df, artists_df).cache()
  # mapped_slice_df.write.parquet(PLAYLIST_EMBEDDINGS)
# else:
  # mapped_slice_df = spark.read.schema(playlist_schema_mapped).parquet(PLAYLIST_EMBEDDINGS)

In [20]:
def get_all_songs(playlist_df: DataFrame) -> DataFrame:
   all_songs = playlist_df.select(explode("tracks.track_uri").alias("track_uri")).distinct()
   return all_songs

def create_songs_pos_mapping(songs_df: DataFrame) -> Tuple[DataFrame, int]:
  songs_df = get_all_songs(slice_df)
  songs_df.createOrReplaceTempView("SONGS_INFO")

  songs_info_df = spark.sql("""
  SELECT 
      row_number() OVER (
          PARTITION BY '' 
          ORDER BY '' 
      ) as pos,
      *
  FROM 
      SONGS_INFO
  """)

  songs_df = songs_info_df.sort("track_uri")
  RATING_VECTOR_LENGTH = songs_df.count()

  return songs_df, RATING_VECTOR_LENGTH


#TODO: Since the .rdd is very slow, I can embed the position information of the track inside the track itself,
# So then I can just do pos_list.add(row.rating_position) in a few miliseconds. 
def map_track_df_to_pos(playlist_df: DataFrame, mapping: DataFrame) -> DataFrame:
    """
    Returns a DataFrames containing the playlists, but the tracks are represented as a binary sparse vector.
    """

    @udf(returnType=VectorUDT())
    def extract_vector(tracks):
      pos_list = set()

      def reduce_fn(pos_list, row):
          pos_list.add(track_uri_to_id.get(row.track_uri))
          return pos_list
      
      pos_list = reduce(reduce_fn, tracks, pos_list)
      
      return SparseVector(RATING_VECTOR_LENGTH + 1, sorted(list(pos_list)), [1 for _ in pos_list])

    # Apply the mapping UDF on the "tracks" column of the slice_df dataframe
    mapped_df = playlist_df.withColumn('tracks', extract_vector(col('tracks')))

    return mapped_df

songs_df, RATING_VECTOR_LENGTH = create_songs_pos_mapping(slice_df)
track_uri_to_id = songs_df.select('track_uri', 'pos').rdd.collectAsMap() # TODO: Pass it as a parameter maybe?
mapped_slice_df = map_track_df_to_pos(slice_df, songs_df).cache()

# Convert PySpark DataFrame into PyTorch DataLoader

In [21]:
import numpy as np
from pyspark.sql.functions import udf, length
from pyspark.ml.linalg import VectorUDT
import torch
from typing import Tuple, List


def convert_sparse_to_indices(df: DataFrame) -> DataFrame:
  """
  Given a dataframe fo columns "pos":int and "tracks":SparseVector, it returns a new dataframe where
  the SparseVector are replaced with a list of the indices where the values are.
  (The value information is lost, but we don't care since they are binary values so they will be all ones)
  """
  # SparseTupleType = ArrayType(
  #   ArrayType(IntegerType())
  # )

  @udf(returnType=ArrayType(IntegerType()))
  def transform_array(item: SparseVector):
    """
    Given a SparseVector (binary) it returns the tuple that represent it, of the type (size, indices)
    """
    indices_list = item.indices.tolist()
    padding_width = max_songs - len(indices_list)
    return indices_list + [-1] * padding_width
  
  max_songs = mapped_slice_df.select(F.max("num_tracks")).first()[0]
  print(f"Max number of songs: {max_songs}")
  df = df.withColumn("embedding_indices", transform_array(col("tracks"))).drop("tracks")
  return df


In [22]:
def padded_tensors_to_sparse_matrix(padded_tensor: torch.Tensor, shape: tuple) -> torch.Tensor:
  batch_size, max_songs = padded_tensor.size(0), padded_tensor.size(1)
  rows = []
  for row_idx in range(batch_size):
    row = padded_tensor[row_idx]
    indices = row[row != -1]
    sparse_tensor = torch.sparse_coo_tensor(indices.unsqueeze(0), torch.ones(indices.shape), shape)
    rows.append(sparse_tensor)
  return torch.stack(rows)

def padded_tensors_to_dense_matrix(padded_tensor: torch.Tensor, shape: tuple) -> torch.Tensor:
  batch_size, max_songs = padded_tensor.size(0), padded_tensor.size(1)
  rows = []
  for row_idx in range(batch_size):
    row = padded_tensor[row_idx]
    indices = row[row != -1]
    sparse_tensor = torch.sparse_coo_tensor(indices.unsqueeze(0), torch.ones(indices.shape), shape)
    dense = sparse_tensor.to_dense()
    rows.append(dense)
  unpadded = torch.stack(rows)
  return unpadded

In the paper they have two matrices,l et $n$ be the number of unique songs, $m$ the number of playlists and $k$ the number of unique artists:

- $P \in \mathbb{R}^{m \times n}$ where $p_i = 1$ if song $i$ is in the playlist, $p_i=0$ otherwise
- $A \in \mathbb{R}^{m \times k}$ where $a_i=1$ if the artist is present in the playlist, $a_i = 0$ otherwise

In [23]:
from petastorm.spark import SparkDatasetConverter, make_spark_converter
from petastorm.unischema import dict_to_spark_row, Unischema, UnischemaField
from petastorm.codecs import ScalarCodec, CompressedImageCodec, NdarrayCodec
from torchvision import transforms
from petastorm import TransformSpec

CACHE = os.path.join(GDRIVE_DIR, "/big_data/cache_5")
spark.conf.set(SparkDatasetConverter.PARENT_CACHE_DIR_URL_CONF, f'file://{CACHE}')

pytorch_songs_df = convert_sparse_to_indices(mapped_slice_df.select("tracks"))
songs_converter = make_spark_converter(pytorch_songs_df)

pytorch_artists_df = convert_sparse_to_indices(artists_slice_df.select("tracks"))
artist_converter = make_spark_converter(pytorch_songs_df)

Max number of songs: 250
Max number of songs: 250


# PyTorch Model

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pickle


class DAE_tied(nn.Module):
    def __init__(self, conf):
        super(DAE_tied, self).__init__()
        self.save_dir = conf["save"]

        self.n_batch = conf["batch"]
        self.n_input = conf["n_input"]
        self.n_hidden = conf["hidden"]
        self.learning_rate = conf["lr"]
        self.reg_lambda = conf["reg_lambda"]

        self.x_positions = torch.LongTensor()
        self.x_ones = torch.FloatTensor()

        self.y_positions = torch.LongTensor()
        self.y_ones = torch.FloatTensor()

        self.keep_prob = torch.tensor(conf["keep_prob"], dtype=torch.float32)
        self.input_keep_prob = torch.tensor(conf["input_keep_prob"], dtype=torch.float32)

        self.weights = {}
        self.biases = {}
        self.d_params = []

    def init_weight(self):
        self.weights['encoder_h'] = nn.Parameter(torch.FloatTensor(self.n_input, self.n_hidden))
        nn.init.xavier_uniform_(self.weights['encoder_h'])
        self.biases['encoder_b'] = nn.Parameter(torch.FloatTensor(self.n_hidden))
        nn.init.zeros_(self.biases['encoder_b'])
        self.biases['decoder_b'] = nn.Parameter(torch.FloatTensor(self.n_input))
        nn.init.zeros_(self.biases['decoder_b'])
        self.d_params = [self.weights['encoder_h'], self.biases['encoder_b'], self.biases['decoder_b']]

    # Building the encoder
    def encoder(self, x):
        # Encoder Hidden layer with sigmoid activation #1
        layer = torch.add(torch.matmul(x, self.weights['encoder_h']), self.biases['encoder_b'])
        layer = torch.sigmoid(layer)
        layer = torch.nn.functional.dropout(layer, p=1 - self.keep_prob)

        return layer

    # Building the decoder
    def decoder(self, x):
        # Decoder Hidden layer with sigmoid activation #1
        layer = torch.sigmoid(torch.add(torch.matmul(x, self.weights['encoder_h'].t()), self.biases['decoder_b']))
        return layer

    def l2_loss(self):
      encoder_h_l2 = torch.norm(self.weights['encoder_h']) ** 2
      decoder_b_l2 = torch.norm(self.biases['decoder_b']) ** 2
      encoder_b_l2 = torch.norm(self.biases['encoder_b']) ** 2

      l2 = encoder_h_l2 + decoder_b_l2 + encoder_b_l2
      return l2

    def forward(self, x, y):

        # TODO: Take sparse matrix representation instead of dense
        # self.x = x_positxions
        # self.x_ones = x_ones
        # self.y_positions = y_positions
        # self.y_ones = y_ones

        self.x = x.t()
        self.y = y.t()

        # x_sparse = torch.sparse.FloatTensor(self.x_positions.t(), self.x_ones, torch.Size([self.n_batch, self.n_input]))
        # self.x = x_sparse.to_dense()
        # y_sparse = torch.sparse.FloatTensor(self.y_positions.t(), self.y_ones, torch.Size([self.n_batch, self.n_input]))
        # self.y = y_sparse.to_dense()

        x_dropout = torch.nn.functional.dropout(self.x, p= 1 - self.input_keep_prob) # Maybe error
        reduce_sum = torch.sum(x_dropout, dim=1, keepdim=True)
        self.x_dropout = torch.div(x_dropout, reduce_sum + 1e-10)

        encoder_op = self.encoder(self.x_dropout)
        self.y_pred = self.decoder(encoder_op)

        l2 = self.l2_loss()

        L = -torch.sum(self.y * torch.log(self.y_pred + 1e-10) +
                       0.55 * (1 - self.y) * torch.log(1 - self.y_pred + 1e-10), dim=1)
        self.cost = torch.mean(L) + self.reg_lambda * l2

    def save_model(self):
        params = [param.detach().numpy() for param in self.d_params]
        with open(self.save_dir, 'wb') as f:
            pickle.dump(params, f)

            
class DAE(DAE_tied):
    def __init__(self, conf):
        super(DAE, self).__init__(conf)
        self.initval_dir = conf["initval"]

    def init_weight(self):
        if self.initval_dir == 'NULL':
            self.weights['encoder_h'] = nn.Parameter(torch.FloatTensor(self.n_input, self.n_hidden))
            nn.init.xavier_uniform_(self.weights['encoder_h'])
            self.weights['decoder_h'] = nn.Parameter(torch.FloatTensor(self.n_input, self.n_hidden))
            nn.init.xavier_uniform_(self.weights['decoder_h'])
            self.biases['encoder_b'] = nn.Parameter(torch.FloatTensor(self.n_hidden))
            nn.init.zeros_(self.biases['encoder_b'])
            self.biases['decoder_b'] = nn.Parameter(torch.FloatTensor(self.n_input))
            nn.init.zeros_(self.biases['decoder_b'])
        else:
            with open(self.initval_dir, 'rb') as f:
                emb = pickle.load(f)
            self.weights['encoder_h'] = nn.Parameter(torch.FloatTensor(emb[0]))
            self.weights['decoder_h'] = nn.Parameter(torch.FloatTensor(emb[1]))
            self.biases['encoder_b'] = nn.Parameter(torch.FloatTensor(emb[2]))
            self.biases['decoder_b'] = nn.Parameter(torch.FloatTensor(emb[3]))

        self.d_params = [self.weights['encoder_h'], self.weights['decoder_h'],
                         self.biases['encoder_b'], self.biases['decoder_b']]

    def decoder(self, x):
        # Decoder Hidden layer with sigmoid activation #1
        layer = torch.sigmoid(torch.add(torch.matmul(x, self.weights['decoder_h'].t()), self.biases['decoder_b']))
        return layer

    def l2_loss(self):
      encoder_h_l2 = torch.norm(self.weights['encoder_h']) ** 2
      decoder_b_l2 = torch.norm(self.biases['decoder_b']) ** 2
      encoder_b_l2 = torch.norm(self.biases['encoder_b']) ** 2
      decoder_h_l2 = torch.norm(self.weights['decoder_h']) ** 2

      l2 = encoder_h_l2 + decoder_b_l2 + encoder_b_l2 + decoder_h_l2
      return l2

I'm trying to figure out what x and y are. From the source code, we can see this in the `data_readery.py:48` file:
```python
trk_positions = np.concatenate(trk_positions)
art_positions = np.concatenate(art_positions)
y_positions = np.concatenate((trk_positions, art_positions), 0)
```
So I can assume that y is just the concatenation of p_i and a_i.

On the other hand, from `data_reader.py:250`, we can see:
```python
trk_positions = np.concatenate(trk_positions)
art_positions = np.concatenate(art_positions)
x_positions = np.concatenate((trk_positions, art_positions), 0)
```

So From this I assume `y` and `x` are the same thing, meanin the concatenation of A and P.

In [28]:
#Hyperparameters used in the paper
NUM_PLAYLISTS = 10_000
conf = {
    'batch': 32,
    'n_input': RATING_VECTOR_LENGTH + ARTIST_VECTOR_LENGTH,
    'hidden': 64,
    'lr': 0.001, #original 0.001
    'reg_lambda': 0.001,
    'initval': 'NULL',
    "keep_prob": 0.8,
    "input_keep_prob": 0.8,
    'save': './'
}
dae_model = DAE(conf)
dae_model.init_weight()
optimizer = optim.Adam(dae_model.d_params, lr=conf['lr'])

In [29]:
from tqdm.notebook import tqdm
NUM_EPOCHS = 10
with songs_converter.make_torch_dataloader(num_epochs = NUM_EPOCHS) as songs_dataloader:
  with artist_converter.make_torch_dataloader(num_epochs = NUM_EPOCHS) as artists_dataloader:
    losses = []
    ARTIST_SHAPE = (ARTIST_VECTOR_LENGTH, )
    SONG_SHAPE = (RATING_VECTOR_LENGTH, )
    # for epoch in tqdm(range(num_epochs), desc="Training..."):
    zipped_dataloaders = zip(songs_dataloader, artists_dataloader)
    for batch_idx, (song, artist) in tqdm(enumerate(zipped_dataloaders), desc=f"Training model", total= (NUM_PLAYLISTS / 32) * NUM_EPOCHS):
      padded_song_tensor = song["embedding_indices"]
      padded_artist_tensor = artist["embedding_indices"]
      
      song_dense = padded_tensors_to_dense_matrix(padded_song_tensor, SONG_SHAPE)
      artist_dense = padded_tensors_to_dense_matrix(padded_artist_tensor, ARTIST_SHAPE)

      optimizer.zero_grad()
      x = torch.concat((song_dense, artist_dense), dim=1).t()
      y = torch.concat((song_dense, artist_dense), dim=1).t()
      dae_model(x, y)
      loss = dae_model.cost
      dae_model.cost.backward()
      optimizer.step()

      if batch_idx % 100 == 0:
        print(f"Loss: {loss}")


Training model:   0%|          | 0/3125.0 [00:00<?, ?it/s]

Loss: 81554.84375
Loss: 10480.935546875
Loss: 4093.239501953125
Loss: 2651.362060546875
Loss: 1879.93798828125
Loss: 1749.5504150390625
Loss: 1353.27978515625
Loss: 1458.247314453125
Loss: 1289.91650390625
Loss: 1383.0006103515625
Loss: 1192.9573974609375
Loss: 1267.356201171875
Loss: 1131.73486328125
Loss: 1235.7421875
Loss: 1063.07666015625
Loss: 1160.0535888671875
Loss: 1114.031005859375
Loss: 1064.7171630859375
Loss: 1153.3277587890625
Loss: 1407.975830078125
Loss: 1140.4365234375
Loss: 1139.4649658203125
Loss: 1240.2803955078125
Loss: 1298.5384521484375
Loss: 1132.6448974609375
Loss: 1427.786376953125
Loss: 1093.15087890625
Loss: 1065.274658203125
Loss: 1249.514892578125
Loss: 1071.7154541015625
Loss: 1227.5726318359375
Loss: 1055.323974609375
